In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv('data/train', sep='\t')

In [3]:
train = train[:20000].dropna()


In [4]:
train

,CLIENT_ID,RETRO_DT,tokens,DEF,urls_hashed
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...
...,...,...,...,...,...
19995,5424938,20200408,займы 3 loan 1 деньги 1 kredito 1 зимний 3 кар...,1,9111377e394acb4f5ec7ef1991fb28b5 1 89eed7fb016...
19996,5428130,20200409,самовывозом 1 телешоу 1 мобильные 1 видеохости...,0,99be73cd2a77189126ff02119a897ba7 1 cd4192c564c...
19997,5474592,20200421,выплату 1 снять 1 zdorovya 1 cruiser 2 svyazi ...,0,4aafb7e452abfdab09c834eceffc4aff 3 d8877075c35...
19998,5337694,20200321,showrecipe 1 arestovali 2 вкусных 1 suburban 2...,0,0e989f11b85cbae272b0d509f19949ce 1 a09034648a7...


In [14]:
%pip install gensim pyLDAvis matplotlib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install pyLDAvis.gensim gensim.corpora gensim.models 

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pyLDAvis
#import pyLDAvis.gensim #don't skip
import matplotlib.pyplot as plt

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel


%matplotlib inline

/home/jupyter-admin/.local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jupyter-admin/.local/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [10]:
import tqdm

In [23]:
train.tokens[0]

'форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 массаж 1 мачехи 1 kakaya 1 tovari 1 дмб 1 milf 2 istoriya 1 парня 1 хуя 1 voyska 1 cvet 1 рода 1 получает 1 слушай 1 molodym 1 porno 3 телепрограмма 1 видит 1 полоса 1 красивыми 1 сделал 1 горячем 1 мама 1 колледже 1 замужняя 1 varit 1 kendra 1 brigada 1 луна 1 цвет 1 lust 1 зрелой 1 зависимости 1 мужиками 1 работу 1 тельняшка 1 sportivnye 1 баба 1 тельняшки 1 завтра 1 зрелых 2 молодых 1 американский 1 качестве 2 дом 1 отличаются 1 заставила 1 мать 1 видео 8 история 1 clothes 1 активация 1 zavisimosti 1 вместо 1 porn 2 неделю 1 летняя 1 бесплатно 1 грудастая 1 предпочитает 1 смотреть 8 пойти 1 video 8 viewmsg 1 молодыми 1 отдельной 1 наказания 1 беспла 1 порно 8 mama 1 негр 1 полная 1 tube 1 массажиста 1 public 1 poshla 1 аппетитную 1 нежный 1 трахается 1 сиськами 1 бригада 1 теплую 1 mature 5 член 1 захотела 1 мамаши 1 форма 1 popular 1 brigady 1 ushli 1 вызвала 1 ranshe 1 genre 1 rus 1 лекси 1 темно 1 расслабиться 1 brat 1 ванну 1 хозяйка

In [47]:
def constants_to_dict(const_value, point):
    dict_const = {}
    for i in range(len(const_value)):
        dict_const[const_value[i]] = point[i]
    return dict_const 
train_dict = []
counter = 0
for i in train.tokens:
    #print(i)
    counter += 1
    if counter%500 == 0:
        print(counter)
    list_tokens = []
    list_count = []
    try:
        for n in range(len(i.split(' '))):
            if n%2 == 0:
        # print(n)
                list_tokens.append(i.split(' ')[n])
            else:
                list_count.append(int(i.split(' ')[n]))
    except Exception:
        print(i)
        pass

    train_dict.append(constants_to_dict(list_tokens, list_count))

KeyboardInterrupt: 

In [48]:
words = []
counter = 0
for i in train.tokens:
    cur_user_words = []
    counter += 1
    if counter%500 == 0:
        print(counter)
    #print(i)
    try:
        for j in range(len(i.split(' '))):
            if j%2 == 0:
        # print(n)
                pass
            else:
                for k in range(int(i.split(' ')[j])):
                    cur_user_words.append(i.split(' ')[j-1])
                #print(i.split(' ')[j-1])
    except Exception:
        print(i, j)
        pass
    words.append(cur_user_words)



500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


In [2]:
data_words[:20]

NameError: name 'data_words' is not defined

In [42]:
def dict_to_list(d):
    data_words = []
    for i in d.items():
        for j in range(i[1]):
            data_words.append(i[0])
    return data_words
data_words = []
for tr_d in tran_dict:
    data_words.append(dict_to_list(tr_d))

In [ ]:
#создадим биграмы
bigram = gensim.models.Phrases(words, min_count=5, threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
#сделаем биграмы
data_words = [bigram_mod[doc] for doc in words]
print(data_words[1])
#объявим два основных параметра LDA
#словарь
id2word = corpora.Dictionary(data_words)
#и перейдем к частоте терминов в документе
corpus = [id2word.doc2bow(text) for text in data_words]

['platiza_мобильные', 'мобильные', 'sergej', 'code_code', 'code_отказов', 'отказов_отказов', 'самовывозом', 'pesen_pesen', 'статья', 'global', 'косметики_продолжительность', 'nalichnimi', 'праздники', 'vysokoy', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'деньги_деньги', 'дружить_дружить', 'microloans_microloans', 'microloans_microloans', 'microloans_microloans', 'microloans_microloans', 'microloans', 'davaj', 'майские_майские', 'analiza', 'авиабилеты', 'операции_операции', 'операции', 'смартфон', 'сумму', 'пасху', 'омскую', 'лекарства', 'займах', 'wizard', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет', 'кабинет

In [50]:
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 4), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 3), (109, 1), (110, 1),

In [52]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20, 
                                            random_state=42,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

CPU times: user 1h 13min 29s, sys: 1h 57min 50s, total: 3h 11min 19s
Wall time: 26min 17s


In [54]:
lda_model.print_topics()

[(0,
  '0.035*"карту" + 0.027*"кредит_кредит" + 0.026*"credit_credit" + 0.022*"заявка_заявка" + 0.018*"банк_банк" + 0.016*"наличными_наличными" + 0.014*"банки_банки" + 0.014*"оформить_оформить" + 0.013*"кредиты_кредиты" + 0.012*"взять_взять"'),
 (1,
  '0.021*"купить" + 0.020*"порно_порно" + 0.016*"авто_авто" + 0.014*"видео_видео" + 0.013*"цены_цены" + 0.013*"video_video" + 0.013*"россии" + 0.012*"продажа_продажа" + 0.012*"auto_auto" + 0.011*"пробегом_пробегом"'),
 (2,
  '0.425*"календарь_календарь" + 0.412*"задачи_задачи" + 0.027*"landing_landing" + 0.013*"day_day" + 0.006*"календарь" + 0.006*"задачи" + 0.003*"landing" + 0.002*"tutorial_tutorial" + 0.001*"security" + 0.001*"recovery"'),
 (3,
  '0.007*"бесплатно" + 0.007*"купить" + 0.003*"фото_фото" + 0.002*"смотреть" + 0.002*"amp_amp" + 0.002*"ответы_ответы" + 0.002*"видео" + 0.002*"video" + 0.002*"цены" + 0.002*"фото"'),
 (4,
  '0.098*"купить" + 0.026*"магазине_магазине" + 0.022*"москве" + 0.020*"магазин_магазин" + 0.014*"цены_цены" +

In [6]:
seq = np.load('data/sequences.npy', allow_pickle=True)

In [7]:
seq = list(seq)[:20000]

In [4]:
tokens = train.tokens

In [7]:
token_words = []
for user in tokens:
    users_data = user.split(" ")
    one_user_data = []
    #print(users_data)
    for i in range(0, len(users_data) - 1, 2):
        if (int(users_data[i+1]) <=14) and (int(users_data[i+1]) >= 4):
            for k in range(int(users_data[i+1])):
                one_user_data.append(users_data[i])
            token_words.append(one_user_data)
        else:
            pass
            


In [8]:
len(token_words)

1602985

In [ ]:
from gensim.models import Word2Vec
w2v_tokens = Word2Vec(token_words, vector_size=500, sg=0, window=5, min_count=15)

In [ ]:
def find_vec(X_train):
    vec = np.zeros(500)
    counter = -1
    try:
        for i in X_train.split():
            counter += 1
            if i in words and counter%2 == 0:
                #print(i)
                vec += w2v.wv.get_vector(i)
            else:
                pass
    except Exception:
        print(i)
    return vec

In [ ]:
vec_tokens = train['tokens'][:20000].dropna().apply(find_vec)
print(vec.head())
#title_data = pd.DataFrame.from_records(title_vec.values, index = title_vec.index)
vec_tokens_data = pd.DataFrame.from_records(vec_tokens.values, index = vec_tokens.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_tokens = pd.concat([train, vec_tokens_data], axis = 1, sort = False)

In [9]:

#нам нужна колонка со списком оставшихся слов без букв
#seq_text = [sent.split() for sent in seq]
#print(matrix_text)
#CBOW is used when sg=0
w2v = Word2Vec(seq[:20000], vector_size=500, sg=0, window=5)

/home/jupyter-admin/.local/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
words = list(w2v.wv.index_to_key)

In [11]:
len(words)

6644

In [12]:
vec_hashes = train['urls_hashed'][:20000].dropna().apply(find_vec)
print(vec.head())
#title_data = pd.DataFrame.from_records(title_vec.values, index = title_vec.index)
vec_hashes_data = pd.DataFrame.from_records(vec_hashes.values, index = vec_hashes.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_hashes = pd.concat([train, vec_hashes_data], axis = 1, sort = False)

0    [0.4349914025515318, 1.5845880657434464, 2.513...
1    [4.150917867722455, 8.687634985893965, 11.5403...
2    [1.1232379330322146, 2.3567246943712234, 3.774...
3    [0.8240354621666484, 3.4220511093735695, 4.030...
4    [13.003102393777226, 10.65344627131708, 21.212...
Name: urls_hashed, dtype: object


NameError: name 'res2' is not defined

In [14]:
res2[:20000].drop(columns=['tokens', 'urls_hashed']).to_csv('data/df_with_seq.csv')

In [25]:
train['urls_hashed'][15000:20000]

15000    38f727b363acaf5da8d07ac1d0f78a1d 1 aee71c8d18d...
15001    b03e2ea2c39ee4ab8a9fdfe1a138988d 1 7f30e505bf8...
15002    aee71c8d18d6b91b36bf599208a91777 1 0be9a879c36...
15003    83e85bee0d81c0e9e5047e202cddf534 1 c8abbde9b65...
15004    5d24dd09d5e197da92f84c154ccbdd62 1 aee71c8d18d...
                               ...                        
19995    9111377e394acb4f5ec7ef1991fb28b5 1 89eed7fb016...
19996    99be73cd2a77189126ff02119a897ba7 1 cd4192c564c...
19997    4aafb7e452abfdab09c834eceffc4aff 3 d8877075c35...
19998    0e989f11b85cbae272b0d509f19949ce 1 a09034648a7...
19999    b58d1eb5070747f7e7bf3733b97389bc 1 aee71c8d18d...
Name: urls_hashed, Length: 5000, dtype: object

In [21]:
train['urls_hashed'][:20000].dropna()

0        7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1        1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
2        6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
3        1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
4        83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...
                               ...                        
19995    9111377e394acb4f5ec7ef1991fb28b5 1 89eed7fb016...
19996    99be73cd2a77189126ff02119a897ba7 1 cd4192c564c...
19997    4aafb7e452abfdab09c834eceffc4aff 3 d8877075c35...
19998    0e989f11b85cbae272b0d509f19949ce 1 a09034648a7...
19999    b58d1eb5070747f7e7bf3733b97389bc 1 aee71c8d18d...
Name: urls_hashed, Length: 19964, dtype: object